<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/06b_HuggingFaceTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[**Hugging Face**](https://huggingface.co/docs) es un ecosistema de librerías que permite a desarrolladores y científicos compartir y usar recursos open-source de machine learning. Es particularmente popular en el campo del NLP.

Vamos a hacer un _overview_ de los principales componentes de Hugging Face: tokenizadores, modelos, datasets y pipelines.

Muchos de estos componentes son interfaces de alto nivel que por debajo usan pytorch.

-----------------------

Tarea: responder donde dice **PREGUNTA**

## Configuración del entorno

In [ ]:
!pip install -qU transformers datasets watermark

In [ ]:
%load_ext watermark

In [ ]:
%watermark -vmp transformers,datasets,torch,numpy,pandas

## Tokenizadores

Los modelos preentrenados se desarrollan junto con **tokenizadores**: toman strings sin procesar y devuelven diccionarios con los **inputs del modelo**.

Cada **token** es un número entero que se corresponde con una **palabra en el vocabulario** del modelo.

Con `AutoTokenizer` podemos cargar **tokenizers pre-entrenados**. Para ver cómo entrenar un tokenizador con BPE desde cero, ver: https://huggingface.co/learn/nlp-course/en/chapter6/8?fw=pt

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [ ]:
print(tokenizer)

In [ ]:
input_str = "These pretzels are making me thirsty!"
tokenized_input = tokenizer(input_str)

print("> Tokenizer input:")
print(input_str)
print("-"*70)
print("> Tokenizer output:")
print(tokenized_input)
print("-"*70)
print("> Tokenizer output (input IDs):")
print(tokenized_input["input_ids"])

Veamos lo que sucede por debajo paso a paso

In [ ]:
def tokenize_step_by_step(input_str):
    input_tokens = tokenizer.tokenize(input_str)
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    cls = [tokenizer.cls_token_id]
    sep = [tokenizer.sep_token_id]
    input_ids_special_tokens = cls + input_ids + sep
    decoded_str = tokenizer.decode(input_ids_special_tokens)
    print("input:                  ", input_str)
    print("tokenize:               ", input_tokens)
    print("convert_tokens_to_ids:  ", input_ids)
    print("add special tokens:     ", input_ids_special_tokens)
    print("-"*70)
    print("decode (IDs to strings):", decoded_str)

tokenize_step_by_step(input_str)

**PREGUNTA 9** ¿Qué cambia cuando pasamos una oración en castellano o árabe vs inglés? ¿Por qué?

In [ ]:
x = "Quiero aprender a usar modelos de lenguaje"

tokenize_step_by_step(x)

In [ ]:
x = "أريد أن أتعلم استخدام نماذج اللغة"

tokenize_step_by_step(x)

Cuando entrenamos modelos o hacemos inferencia, vamos a querer:

* trabajar con **_batches_**, pasando muchas secuencias simultáneamente como input
* trabajar con **tensores** de PyTorch, no con listas

**PREGUNTA 10** ¿para qué sirve usar batches?

In [ ]:
input_strings = [
    "These pretzels are making me thirsty!",
    "I am speechless! I am without speech.",
    "No more soup for you!",
    "I'm a wealthy industrialist and philanthropist and a bicyclist."
]

In [ ]:
model_inputs = tokenizer(
    input_strings, return_tensors="pt", padding='longest', truncation=True,
    max_length=tokenizer.model_max_length)

Lo que hicimos recién es:

- Tokenizar todas las frases
- Devolver los tensores en formato PyTorch ("pt") en un tensor _rectangular_
- Truncar las frases más largas para que no excedan el tamaño máximo admitido por el modelo
- Rellenar con _padding_ hasta el máximo largo del batch para que todas las entradas tengan la misma longitud

Cuando entrenamos modelos o hacemos inferencia, a veces es recomendable tokenizar cada batch on-the-fly en lugar de pretokenizar -- esto permite experimentar más rápido con muchos datos.

In [ ]:
print(f"Max model length: {tokenizer.model_max_length}")

In [ ]:
print(f"Pad token: {tokenizer.pad_token}")
print(f"Pad token ID: {tokenizer.pad_token_id}")

In [ ]:
model_inputs = tokenizer(
    input_strings, return_tensors="pt", padding='longest', truncation=True,
    max_length=tokenizer.model_max_length)

print("Batch encode:")
print([f"{k}: {v.shape}" for k, v in model_inputs.items()])
print(model_inputs["input_ids"])
print(model_inputs["attention_mask"])
print("-"*70)
print("Batch decode:")
print(*tokenizer.batch_decode(model_inputs.input_ids, skip_special_tokens=False), sep="\n")

## Modelos

Los modelos suelen tener un **body** y **head**.

* El "body" son los **pesos preentrenados** que devuelven una representación de la secuencia de input.
* El "head" son los pesos adicionales que dependen de la **tarea específica** que estamos resolviendo.

Con las clases `AutoModel...` podemos cargar un modelo preentrenado y agregarle un head específico para nuestra tarea.

```
AutoModel # (solo hidden states, sin head)
AutoModelForCausalLM
AutoModelForMaskedLM
AutoModelForSequenceClassification
AutoModelForTokenClassification
# etc
```

Vamos a cargar un BERT "destilado" para hacer **clasificación binaria de secuencias**.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-cased', num_labels=2, id2label={0: "A", 1: "B"}, label2id={"A": 0, "B": 1})

Esto quiere decir que estamos agregando una **capa de clasificación** con **dos salidas** al final del modelo preentrenado.

El warning nos dice que los pesos de esta capa todavía no fueron entrenados. Es decir, necesitamos hacer fine-tuning sobre un dataset específico para que tenga sentido usarlo.

**_Solo a modo ilustrativo_**, vamos a hacer **inferencia** sobre una frase de ejemplo.

In [ ]:
print(model)

In [ ]:
param_names = [n for n, p in model.named_parameters()]

print(f"# de 'capas': {len(param_names)}")
print(param_names[:3])
print(param_names[-3:])

In [ ]:
import torch

input_str = "These pretzels are making me thirsty!"

model_inputs = tokenizer(input_str, return_tensors="pt")
model.eval() # eval mode: desactiva componentes random, como dropout
with torch.inference_mode(): # inference mode: desactiva cómputo de gradientes
    model_outputs = model(**model_inputs)

print("Inputs:")
print(model_inputs)
print("-"*70)
print("Outputs:")
print(model_outputs)
print(f"Logits: {model_outputs.logits}")
print(f"Probabilidades: {torch.softmax(model_outputs.logits, dim=1)}")
pred = torch.argmax(model_outputs.logits).item()
print(f"Predicción: {model.config.id2label[pred]}")

Si tuviésemos labels, podemos usar pytorch para entrenar i.e. actualizar los pesos del modelo para optimizar la loss.

In [ ]:
input_str = "These pretzels are making me thirsty!"

model_inputs = tokenizer(input_str, return_tensors="pt")
model.train()
model_outputs = model(**model_inputs)
label = torch.tensor([1])
loss = torch.nn.functional.cross_entropy(model_outputs.logits, label)
print(f"Loss: {loss.item():.4f}")
loss.backward() # Computa gradientes
# optimizer.step() # Si quisieramos actualizar los pesos con un optimizer

Si los labels están en el input, podemos obtener la loss automáticamente:

In [ ]:
model_inputs['labels'] = torch.tensor([1]) # label de ejemplo
model.eval()
with torch.inference_mode():
    model_outputs = model(**model_inputs)

print(f"Logits: {model_outputs.logits}")
print(f"Probabilidades: {torch.softmax(model_outputs.logits, dim=1)}")
print(f"Loss: {model_outputs.loss:.4f}")

**PREGUNTA 11** ¿por qué puede diferir la loss en las dos celdas anteriores?

## Datasets

HF también tiene [datasets](https://huggingface.co/datasets) open-source que podemos usar para entrenar y evaluar nuestros modelos.

Hay [muchas funcionalidades](https://huggingface.co/docs/datasets/process) para leer y modificar la estructura y contenido de un dataset (e.g. streaming, split de datos, reordenar filas, cambiar nombres de columnas, eliminar columnas, transformar ejemplos, concatenar datasets, etc.)

Vamos a cargar un dataset de [reviews de películas](https://huggingface.co/datasets/rotten_tomatoes).

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")

In [ ]:
# filas y columnas
dataset

In [ ]:
dataset["train"].features.items()

In [ ]:
dataset["train"][3]

In [ ]:
# En general es útil armar una función para mapear de IDs a labels de la variable respuesta
label_names = dataset["train"].features["label"].names
label2id = {name: dataset["train"].features["label"].str2int(name) for name in label_names}
id2label = {id: label for label, id in label2id.items()}

id_example = dataset["train"][3]["label"]
print(f"Label ID: {id_example}")
print(f"Label: {id2label[id_example]}")

A modo de ejemplo, vamos a limpiar cualquier caracter HTML que pueda haber en las reviews y truncar.

In [ ]:
import html

def truncate(examples, max_length=50):
    """Recibe un diccionario con los nombres de las columnas como keys
    Como lo vamos a aplicar en batches, cada value del dict es una lista con los
    valores de esa columna
    """
    return {
        'text': [html.unescape(text[:max_length]) for text in examples['text']],
        # 'label': ... # si quisieramos modificar el label
    }

In [ ]:
# ejemplo:
truncate(dataset["train"][:4])

In [ ]:
dataset = dataset.map(lambda x: truncate(x, max_length=50), batched=True)
# batch_size default es 1000

In [ ]:
dataset

In [ ]:
dataset['train'][3]

## Pipelines

Hay tareas estándar de NLP para las que ya hay **modelos preentrenados y fine-tuneados**. HF los disponibiliza a través de la interfaz de [pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines).

Por ejemplo para **clasificación de sentimiento**:

In [ ]:
from transformers import pipeline

sentiment_analysis = pipeline(
    "sentiment-analysis", model="siebert/sentiment-roberta-large-english"
)

In [ ]:
sentiment_analysis("Change is inevitable")

In [ ]:
sentiment_analysis("Change is inevitable", top_k=None) # devuelve los scores de todas las clases

In [ ]:
input_strings = [
    "These pretzels are making me thirsty!",
    "I am speechless! I am without speech.",
    "No more soup for you!",
    "I'm a wealthy industrialist and philanthropist and a bicyclist."
]
outputs = sentiment_analysis(input_strings)

for i, output in enumerate(outputs):
    print(f"Input: {input_strings[i]}")
    print(f"Sentiment: {output['label']}, score: {output['score']:.4f}")
    print("-"*70)

O para [NER](https://huggingface.co/dslim/bert-base-NER):

In [ ]:
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
ner = pipeline("ner", model="dslim/bert-base-NER", tokenizer="dslim/bert-base-NER")

In [ ]:
print(ner.model.config.id2label)

In [ ]:
ner_string = (
    "In Mendoza, José de San Martín met with representatives of the Supreme Court"
    " of Argentina after the Boca Juniors victory in the Copa Libertadores, while"
    " the Ministry of Culture, J. Mendoza, highlighted the influence of tango as a UNESCO"
    " Intangible Cultural Heritage in San Martín, Buenos Aires."
)

In [ ]:
outputs = ner(ner_string)
for entity in outputs:
    print(entity)